[View in Colaboratory](https://colab.research.google.com/github/kmanicka/CarND-Behavioral-Cloning-P3/blob/master/Model.ipynb)

# Udacity Car ND : Behavioral Cloning Project

This notebook is used to build and train the model used for driving the car autonomously 


##  Workspace setup

* clean the workspace 
* download and unzip files. 
* list and assert files

In [2]:
!date > log.txt
!rm -rf *.zip  >> log.txt 2>&1
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip >> log.txt 2>&1
!unzip data.zip >> log.txt 2>&1
!date >> log.txt 2>&1



data  data.zip	log.txt  __MACOSX
driving_log.csv  IMG
  24108   24108  819672


In [45]:
!ls
!ls data
!ls data/IMG | wc

data  data.zip	log.txt  __MACOSX
driving_log.csv  IMG
  24108   24108  819672


## Load Data 

* load image
* load csv file 


In [54]:
import numpy as np
import pandas as pd
from skimage.io import imread_collection

images_raw = imread_collection('data/IMG/center*',conserve_memory=True)
driving_log=pd.read_csv('data/driving_log.csv', sep=',')

driving_log.describe()

,steering,throttle,brake,speed
count,8036.000000,8036.000000,8036.000000,8036.000000
mean,0.004070,0.869660,0.001970,28.169839
std,0.128840,0.301326,0.036565,6.149327
min,-0.942695,0.000000,0.000000,0.502490
25%,0.000000,0.985533,0.000000,30.183093
50%,0.000000,0.985533,0.000000,30.186400
75%,0.000000,0.985533,0.000000,30.186640
max,1.000000,0.985533,1.000000,30.709360


## Prepare Data

* X_train as the images 
* Y_train as the steering

In [55]:
X_train = np.array(images_raw)


Y_train = driving_log['steering']
Y_train = np.reshape(Y_train, (Y_train.shape[0],1))

train_n = X_train.shape[0]
image_height = X_train.shape[1]
image_width = X_train.shape[2]
image_channels = X_train.shape[3]

print('Number of training samples ',train_n)
print('Image Height ',image_height)
print('Image Width ',image_width)
print('Image Channels ',image_channels)

print(X_train.shape)
print(Y_train.shape)


Number of training samples  8036
Image Height  160
Image Width  320
Image Channels  3
(8036, 160, 320, 3)
(8036, 1)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [48]:
# Normalize Training

print(X_train[0,0,0])
# X_train = (X_train - 128.) / 128.  # this is not working. causing memory error. 
print(X_train[0,0,0])

[130 163 208]
[130 163 208]


# Build and Train Model

* Build 
* Train 

In [63]:
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D



#def get_model_base(input_shape) :
  
#  model = Sequential()
#  model.add(Flatten(input_shape=input_shape))
#  model.add(Dense(1))
  
#  model.compile(loss='mse', optimizer='adam')
  
#  return model


def get_model_LeNet(input_shape) :
  
  model = Sequential()

  #Layer 1 
  model.add(Conv2D(4, (5, 5), padding="valid",input_shape=input_shape))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

  #Layer 2
  model.add(Conv2D(8, (5, 5), padding="valid"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  #model.add(Dropout(0.25))
  
  
  #Layer 2.A
  model.add(Conv2D(16, (5, 5), padding="valid"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  #model.add(Dropout(0.25))
  
  model.add(Flatten())
  
  #Layer 3
  model.add(Dense(32))
  model.add(Activation("relu"))
  #model.add(Dropout(0.25))
  
  #Layer 4
  model.add(Dense(8))
  model.add(Activation("relu"))
  #model.add(Dropout(0.25))

  #Layer 5
  model.add(Dense(1)) 
  
  
  model.compile(loss='mse', optimizer='adam')
  
  
  return model



model = get_model_LeNet((image_height,image_width,image_channels))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 156, 316, 4)       304       
_________________________________________________________________
activation_77 (Activation)   (None, 156, 316, 4)       0         
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 78, 158, 4)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 74, 154, 8)        808       
_________________________________________________________________
activation_78 (Activation)   (None, 74, 154, 8)        0         
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 37, 77, 8)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 33, 73, 16)        3216      
__________

In [64]:
from keras.callbacks import ModelCheckpoint
  

filepath="weights-2-{epoch:02d}.hdf5"

checkpoint = ModelCheckpoint(filepath, verbose=1)

history = model.fit(X_train, Y_train, 
                    validation_split=0.2,
                    shuffle=True, 
                    epochs=10, 
                    batch_size=2,
                    callbacks=[checkpoint])



Train on 6428 samples, validate on 1608 samples
Epoch 1/10


KeyboardInterrupt: ignored

In [34]:
!ls -lrt

total 328820
drwxr-xr-x 3 root root      4096 Dec 13  2016 data
drwxrwxr-x 3 root root      4096 Dec 13  2016 __MACOSX
-rw-r--r-- 1 root root 333137665 Dec 13  2016 data.zip
-rw-r--r-- 1 root root   3557136 May 24 06:41 log.txt


In [39]:
#!rm weights*

## Download Model 

In [17]:
from google.colab import files
files.download('modelv5_3_32.h5')

KeyboardInterrupt: ignored